In [1]:
import sys
#sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
from functions import *

In [2]:
def get_resids(data, var, formula):
    import statsmodels.formula.api as smf
    model = smf.ols(var + formula, data=data)
    model = model.fit()
    resid = model.resid
    return resid

def add_resid(data, var, formula):
    data[var + "_resid"] = get_resids(data, var, formula)
    return data

full_formula = '~ interview_age + C(sex)'
no_sex_formula = '~ interview_age'

In [3]:
dropped_siblings = pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/dropped_siblings.csv')

participants_full = (pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/participants.tsv', sep='\t')
                        .assign(ID = lambda x: x['participant_id'].str.replace('sub-', ''),
                                sex = lambda x: np.where(x['sex'] == 1, 'Male', 'Female'))
                        .drop_duplicates('ID', keep='first')
                    )

#participants_site_scanner_software = participants_full[['ID', 'site', 'scanner_model', 'scanner_software']]

participants = participants_full[['ID', 'matched_group']]
participants_sex = participants_full[['ID', 'sex']]
participants = participants.merge(dropped_siblings, on='ID')

participants.shape

(9854, 2)

In [4]:
sample1 = participants.query('matched_group == 1')
sample2 = participants.query('matched_group == 2')

males = participants_sex.query('sex == "Male"')
females = participants_sex.query('sex == "Female"')

In [5]:
rpath_subcort = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/mrirscor02.txt'


rest_baseline_subcort = readtxt(rpath_subcort)
rest_baseline_subcort['src_subject_id'] = rest_baseline_subcort['src_subject_id'].str.replace('_', '', regex=True)

rest_baseline_subcort_mask = rest_baseline_subcort['eventname'].str.contains('baseline')   
rest_baseline_subcort = rest_baseline_subcort[rest_baseline_subcort_mask]

rest_baseline_subcort = rest_baseline_subcort.drop(['collection_id', 'interview_age', 'mrirscor02_id','dataset_id', 'subjectkey',
                                                    'interview_date', 'eventname','rsfmri_cor_ngd_scs_visitid', 'sex',
                                                    'rsfmri_cor_ngd_scs_tr','rsfmri_cor_ngd_scs_numtrs','rsfmri_cor_ngd_scs_nvols',
                                                    'rsfmri_cor_ngd_scs_stnvols','rsfmri_cor_ngd_scs_stcgnvols','rsfmri_cor_ngd_scs_ntpoints',
                                                    'rsfmri_cor_ngd_scs_meanmn','rsfmri_cor_ngd_scs_maxmn','rsfmri_cor_ngd_scs_meantrans',
                                                    'rsfmri_cor_ngd_scs_maxtrans','rsfmri_cor_ngd_scs_meanrot','rsfmri_cor_ngd_scs_maxrot','collection_title'], axis = 1).rename(columns={"src_subject_id": "ID"})

rest_baseline_subcort.columns = rest_baseline_subcort.columns.str.replace('rsfmri_cor_ngd_', '', regex=True)
rest_baseline_subcort.columns = rest_baseline_subcort.columns.str.replace('_scs_', '_', regex=True) 

rest_baseline_subcort[rest_baseline_subcort.iloc[:, 1:].columns] = rest_baseline_subcort.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

rest_baseline_subcort = pd.merge(rest_baseline_subcort, participants[['ID']], on='ID')

In [6]:
rest_baseline_subcort.shape

(9395, 248)

In [7]:
#rest_creation = '''
rpath = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/abcd_betnet02.txt'

rest_baseline = readtxt(rpath)
rest_baseline['src_subject_id'] = rest_baseline['src_subject_id'].str.replace('_', '', regex=True)

rest_1_mask = rest_baseline['rsfmri_c_ngd_visitid'].str.contains('baseline')   
rest_baseline= rest_baseline[rest_1_mask]

rest_motion = rest_baseline[['src_subject_id', 'rsfmri_c_ngd_tr', 'rsfmri_c_ngd_numtrs',
       'rsfmri_c_ngd_nvols', 'rsfmri_c_ngd_stnvols',
       'rsfmri_c_ngd_stcontignvols', 'rsfmri_c_ngd_ntpoints',
       'rsfmri_c_ngd_meanmotion', 'rsfmri_c_ngd_maxmotion',
       'rsfmri_c_ngd_meantrans', 'rsfmri_c_ngd_maxtrans',
       'rsfmri_c_ngd_meanrot', 'rsfmri_c_ngd_maxrot']].rename(columns={"src_subject_id": "ID"})

rest_baseline = rest_baseline.drop(['collection_id', 'abcd_betnet02_id', 'dataset_id', 'subjectkey','interview_date', 'eventname',
       'rsfmri_c_ngd_visitid', 'rsfmri_c_ngd_tr', 'rsfmri_c_ngd_numtrs',
       'rsfmri_c_ngd_nvols', 'rsfmri_c_ngd_stnvols',
       'rsfmri_c_ngd_stcontignvols', 'rsfmri_c_ngd_ntpoints',
       'rsfmri_c_ngd_meanmotion', 'rsfmri_c_ngd_maxmotion',
       'rsfmri_c_ngd_meantrans', 'rsfmri_c_ngd_maxtrans',
       'rsfmri_c_ngd_meanrot', 'rsfmri_c_ngd_maxrot', 'collection_title'], axis = 1).rename(columns={"src_subject_id": "ID"})

rest_baseline.columns = rest_baseline.columns.str.replace('rsfmri_c_ngd_', '', regex=True)
rest_baseline.columns = rest_baseline.columns.str.replace('_ngd_', '_', regex=True) 

rest_cols_subset = ['ID','interview_age', 'sex', 'ad_ad','ad_cgc','ad_ca','ad_dt','ad_dla',
'ad_fo','ad_n','ad_rspltp','ad_smh','ad_smm','ad_sa','ad_vta','ad_vs','cgc_cgc',
'cgc_ca','cgc_dt','cgc_dla','cgc_fo','cgc_n','cgc_rspltp','cgc_smh','cgc_smm','cgc_sa',
'cgc_vta','cgc_vs','ca_ca','ca_dt','ca_dla','ca_fo','ca_n','ca_rspltp','ca_smh','ca_smm',
'ca_sa','ca_vta','ca_vs','dt_dt','dt_dla','dt_fo','dt_n','dt_rspltp','dt_smh','dt_smm','dt_sa','dt_vta',
'dt_vs','dla_dla','dla_fo','dla_n','dla_rspltp','dla_smh','dla_smm','dla_sa','dla_vta',
'dla_vs','fo_fo','fo_n','fo_rspltp','fo_smh','fo_smm','fo_sa','fo_vta','fo_vs','n_n',
'n_rspltp','n_smh','n_smm','n_sa','n_vta','n_vs','rspltp_rspltp','rspltp_smh','rspltp_smm',
'rspltp_sa','rspltp_vta','rspltp_vs','smh_smh','smh_smm','smh_sa','smh_vta','smh_vs',
'smm_smm','smm_sa','smm_vta','smm_vs','sa_sa','sa_vta','sa_vs','vta_vta','vta_vs','vs_vs']

rest_baseline = rest_baseline[rest_cols_subset]

rest_baseline[rest_cols_subset[3:]] = rest_baseline[rest_cols_subset[3:]].apply(pd.to_numeric, errors='coerce')
rest_baseline['interview_age'] = rest_baseline['interview_age'].apply(pd.to_numeric, errors='coerce')
#rest_baseline = pd.merge(rest_baseline, participants_sex, on = 'ID').dropna().drop_duplicates()
rest_baseline = pd.merge(rest_baseline, participants[['ID']], on='ID')

rest_baseline.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/rest_baseline_combined.csv')
rest_motion.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/rest_baseline_motion_combined.csv')

In [8]:
rest_baseline.shape

(9395, 94)

In [9]:
rest_baseline = pd.merge(rest_baseline, rest_baseline_subcort, on ='ID').dropna()

rest_baseline.shape

(9311, 341)

In [10]:
rest_baseline_resid = rest_baseline.copy()

for i in rest_baseline_resid.drop(['ID', 'sex', 'interview_age'], axis=1).columns:
    rest_baseline_resid = add_resid(rest_baseline_resid, i, full_formula)

In [11]:
rest_baseline_resid = pd.concat([rest_baseline_resid[['ID']],rest_baseline_resid.filter(regex='_resid')], axis=1)
rest_baseline_resid.columns = [col.replace('_resid', '') for col in rest_baseline_resid.columns]
rest_baseline_resid.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/rest_baseline_resid_combined.csv')

In [12]:
#rest_cols = rest_baseline_resid.iloc[:, 1:].columns.to_list()
#rest_outliers = get_anoms(rest_baseline_resid, rest_cols)
#print(rest_outliers.groupby('outliers_sum').count().iloc[:, :1])
#rest_outliers = rest_outliers.query('outliers_sum != -3').drop('outliers_sum', axis=1)

In [13]:
sample1_rest = pd.merge(rest_baseline_resid, sample1, on='ID').drop('matched_group', axis=1)
sample2_rest = pd.merge(rest_baseline_resid, sample2, on='ID').drop('matched_group', axis=1)
#sample1_rest= pd.merge(rest_outliers, sample1, on='ID').drop('matched_group', axis=1)
#sample2_rest= pd.merge(rest_outliers, sample2, on='ID').drop('matched_group', axis=1)

sample1_rest.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/sample1_rest_baseline_07282023_combined.csv')
sample2_rest.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/sample2_rest_baseline_07282023_combined.csv')

In [14]:
full_sample = pd.concat([sample1_rest, sample2_rest])

full_sample.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/fullsample_rest_baseline_07282023_combined.csv')

In [15]:
full_sample

,ID,ad_ad,ad_cgc,ad_ca,ad_dt,ad_dla,ad_fo,ad_n,ad_rspltp,ad_smh,...,vs_vtdclh,vs_crcxrh,vs_thprh,vs_cderh,vs_ptrh,vs_plrh,vs_hprh,vs_agrh,vs_aarh,vs_vtdcrh
0,NDARINV7XVXWCME,0.004512,-0.042449,-0.033510,0.108746,-0.022039,0.042342,0.042819,0.013472,0.026994,...,0.025041,-0.005113,0.036332,0.013432,-0.134492,-0.009480,-0.056875,-0.081868,-0.014411,-0.091312
1,NDARINV8CNEFJ83,-0.023402,0.036956,-0.064275,-0.082853,0.043783,-0.049751,-0.040988,-0.043157,-0.003793,...,-0.043398,0.038758,-0.015622,-0.042696,-0.042127,0.049821,-0.035705,0.010753,-0.151861,0.030856
2,NDARINV8M2Z9BTA,-0.039850,0.010658,0.023005,-0.009680,0.001655,0.007598,-0.018956,-0.035899,-0.045319,...,-0.006933,0.008433,-0.015471,0.061347,0.012156,-0.041291,0.131925,-0.014015,0.053274,0.006857
3,NDARINV8U909UGM,-0.066486,-0.008518,0.323638,0.137863,-0.126652,-0.099831,0.023623,0.069163,-0.059067,...,0.214215,-0.338118,0.003675,0.035426,-0.019092,-0.031991,-0.106355,0.368642,0.236141,-0.115051
4,NDARINVDFY7YYM9,0.035605,-0.001720,-0.046636,0.014803,-0.004349,-0.013489,-0.028033,-0.062066,0.016568,...,-0.010703,-0.016684,-0.011662,0.019448,0.014709,0.095418,0.021428,-0.148041,-0.132325,-0.018327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4509,NDARINVKTN3FFWD,0.093015,0.099794,0.023471,0.032005,-0.061235,-0.021067,-0.016699,-0.052364,0.054967,...,0.102359,-0.048095,0.098307,0.091722,0.065828,0.028403,0.030177,0.167991,0.058047,-0.013243
4510,NDARINVL42FE3NR,-0.000798,-0.019916,0.020230,0.059844,0.018478,0.024117,0.032622,0.042045,-0.014380,...,0.123689,-0.058856,-0.000994,0.034092,-0.077435,-0.039797,-0.099958,-0.082227,0.002321,-0.033033
4511,NDARINVL5JJEUJJ,-0.052271,-0.005109,0.013094,0.034510,0.009541,0.087585,0.032014,0.042570,-0.016928,...,0.033413,-0.001633,0.032833,-0.016595,-0.038892,0.022931,0.120825,0.067783,0.056800,-0.062968
4512,NDARINVLVJCXFZ6,-0.080751,0.008501,0.018981,0.016652,0.038769,-0.005640,-0.041629,-0.069086,-0.069650,...,-0.080120,0.027288,-0.020978,-0.033879,0.019678,0.004170,-0.033947,0.060344,0.084619,0.065513
